In [5]:
# Importar bibliotecas necessárias
import pandas as pd
from conexao_bancodedados import *

In [3]:
# Definir o caminho do arquivo CSV
csv_file = r'D:\Projetos\Jornada_financas_pessoais\data\Controle de Ações e FII - Yasmin - FII.csv'

# Ler o arquivo CSV delimitado por vírgula
df = pd.read_csv(csv_file, sep=',')

In [10]:
# Renomear as colunas do DataFrame para corresponder aos nomes das colunas no MySQL
df = df.rename(columns={
    'DATA': 'data_operacao',  # Substitua pelos nomes das colunas corretos
    'PAPEL': 'codigo_negociacao',
    'OP': 'tipo_operacao',
    'PREÇO': 'preco_ativo',
    'QTD': 'quantidade_negociada',
    'CUS.T.': 'custo_total',
    'TOTAL OP': 'total_operacao',
    'RAT.': 'custo_rateado',
    'V. LIQUIDO': 'valor_liquido',
    'ESTOQUE': 'quantidade_estoque',
})

SyntaxError: invalid syntax (1102527919.py, line 6)

In [9]:
# Exibir as primeiras linhas do DataFrame para confirmar a leitura
print(df.head())

  data_operacao codigo_negociacao tipo_operacao  QTD   PREÇO CUS.T. TOTAL OP  \
0    26/10/2021            LVBI11             C    2   97,78   0,19   195,56   
1    26/10/2021            RBED11             C    2  132,10   0,19   264,20   
2    26/10/2021            TRXF11             C    2  101,21   0,19   202,42   
3    08/11/2021            XPML11             C    6   96,47   0,16   578,82   
4    26/11/2021            HGRU11             C    1  102,65   0,19   102,65   

   RAT. V. LIQUIDO  ESTOQUE   PMEDIO GAN/PER IR MÊS MÊS REF. VENDAS/MÊS  
0  0,05    -195,61        2   97,805     NaN    NaN  10/2021       0,00  
1  0,07    -264,27        2  132,135     NaN    NaN  10/2021       0,00  
2  0,05    -202,47        2  101,235     NaN    NaN  10/2021       0,00  
3  0,16    -578,98        6   96,497     NaN    NaN  11/2021       0,00  
4  0,02    -102,67        1  102,670     NaN    NaN  11/2021       0,00  


In [6]:
# Truncar os dados da tabela antes de carregar
truncate_table('bronze','controle_ativo')

In [7]:
# Conexão com o banco de dados na camada "bronze":
engine = conexao_mysql('bronze')

df.to_sql(name='controle_ativo', con=engine, if_exists='append', index=False)

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1054 (42S22): Unknown column 'DATA' in 'field list'
[SQL: INSERT INTO controle_ativo (`DATA`, `PAPEL`, `OP`, `QTD`, `PREÇO`, `CUS.T.`, `TOTAL OP`, `RAT.`, `V. LIQUIDO`, `ESTOQUE`, `PMEDIO`, `GAN/PER`, `IR MÊS`, `MÊS REF.`, `VENDAS/MÊS`) VALUES (%(DATA)s, %(PAPEL)s, %(OP)s, %(QTD)s, %(PREÇO)s, %(CUS_T_)s, %(TOTAL_OP)s, %(RAT_)s, %(V__LIQUIDO)s, %(ESTOQUE)s, %(PMEDIO)s, %(GAN/PER)s, %(IR_MÊS)s, %(MÊS_REF_)s, %(VENDAS/MÊS)s)]
[parameters: [{'DATA': '26/10/2021', 'PAPEL': 'LVBI11', 'OP': 'C', 'QTD': 2, 'PREÇO': '97,78', 'CUS_T_': '0,19', 'TOTAL_OP': '195,56', 'RAT_': '0,05', 'V__LIQUIDO': '-195,61', 'ESTOQUE': 2, 'PMEDIO': '97,805', 'GAN/PER': None, 'IR_MÊS': None, 'MÊS_REF_': '10/2021', 'VENDAS/MÊS': '0,00'}, {'DATA': '26/10/2021', 'PAPEL': 'RBED11', 'OP': 'C', 'QTD': 2, 'PREÇO': '132,10', 'CUS_T_': '0,19', 'TOTAL_OP': '264,20', 'RAT_': '0,07', 'V__LIQUIDO': '-264,27', 'ESTOQUE': 2, 'PMEDIO': '132,135', 'GAN/PER': None, 'IR_MÊS':